In [2]:
!pip install torch transformers boto3 sagemaker mlflow dagshub s3fs -q

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [44]:
role = os.getenv("AWS_ADMIN_ROLE")

## Set mlflow

In [20]:
remote_server_uri = os.getenv("REMOTE_SERVER_URI")
mlflow.set_tracking_uri(remote_server_uri)
mlflow.pytorch.autolog()

## Get sentences and labels


In [53]:
bucket = os.getenv("S3_BUCKER_NAME")
dataset = 'imdb_dataset.csv'
data = pd.read_csv(f"s3://{bucket}/{dataset}")[:100]

In [54]:
label_dict = dict({'positive': 1, 'negative': 0})
data['sentiment'] = data['sentiment'].map(label_dict)

In [55]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data)
train.to_csv("./data/train.csv", index=False)
test.to_csv("./data/test.csv", index=False)

In [48]:
train.head()

,review,sentiment
5,"Probably my all-time favorite movie, a story o...",positive
10,Phil the Alien is one of those quirky films wh...,negative
25,The Karen Carpenter Story shows a little more ...,positive
2,I thought this was a wonderful way to spend ti...,positive
21,I had the terrible misfortune of having to vie...,negative


# Preprocess Data

In [56]:
inputs_train = sagemaker_session.upload_data("./data/train.csv", bucket=bucket, key_prefix=prefix)
inputs_test = sagemaker_session.upload_data("./data/test.csv", bucket=bucket, key_prefix=prefix)

## Training Script

We use the PyTorch-Transformers library, which contains PyTorch implementations and pre-trained model weights for many NLP models, including BERT.

Our training script should save model artifacts learned during training to a file path called model_dir, as stipulated by the SageMaker PyTorch image. Upon completion of training, model artifacts saved in model_dir will be uploaded to S3 by SageMaker and will become available in S3 for deployment.

We save this script in a file named train_deploy.py, and put the file in a directory named code/. The full training script can be viewed under code/.

In [57]:
!pygmentize code/train_deploy.py

import argparse
import json
import logging
import os
import sys

import numpy as np
import pandas as pd
import torch
import torch.distributed as dist
import torch.utils.data
import torch.utils.data.distributed
from torch.utils.data import DataLoader, RandomSampler, TensorDataset
from transformers import AdamW, BertForSequenceClassification, BertTokenizer

import mlflow
from sklearn.metrics import f1_score

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

MAX_LEN = 1024  # this is the max length of the sentence

print("Loading BERT tokenizer...")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)


def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


def _get_train_data_loader(batch_size, training_dir, is_distributed):
    logger.info("Get train data loa

# Run training

In [ ]:
from sagemaker.pytorch import PyTorch

# place to save model artifact
output_path = f"s3://{bucket}/{prefix}"

estimator = PyTorch(
    entry_point="train_deploy.py",
    source_dir="code",
    role=role,
    framework_version="2.1.0",
    py_version="py310",
    instance_count=1,  # this script only support distributed training for GPU instances.
    instance_type="ml.m5.4xlarge",
    output_path=output_path,
    hyperparameters={
        "epochs": 5,
        "num_labels": 2,
        "backend": "gloo",
    },
    disable_profiler=True,
)
estimator.fit({"training": inputs_train, "testing": inputs_test})

2024-06-08 09:07:59 Starting - Starting the training job...
2024-06-08 09:08:17 Starting - Preparing the instances for training...
2024-06-08 09:08:52 Downloading - Downloading the training image...
2024-06-08 09:09:22 Training - Training image download completed. Training in progress.....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-06-08 09:09:52,484 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-06-08 09:09:52,485 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-06-08 09:09:52,485 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-06-08 09:09:52,495 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-06-08 09:09:52,497 sagemaker_pytorch_container.training INFO     Invoking user training script.
2024-06-08 09:09:53,997 sagemaker-training-toolki